In [2]:
import os, json, re
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document
from pinecone import Pinecone, ServerlessSpec

# 1. 환경변수 로드
load_dotenv()
OPENAI_API_KEY  = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
INDEX_NAME      = "card-index"

# 2. Pinecone 초기화
pc = Pinecone(api_key=PINECONE_API_KEY)
pinecone_index = pc.Index(INDEX_NAME)

# 3. 임베딩 준비
embedder = OpenAIEmbeddings(model = "text-embedding-3-small", openai_api_key=OPENAI_API_KEY)

# 4. 카드 데이터 로드
with open("cards.json", encoding="utf-8") as f:
    cards = json.load(f)

docs = []

def parse_fee(fee_text):
    """30,000원 -> 30000"""
    if fee_text is None:
        return None
    fee_num = re.sub(r"[^\d]", "", fee_text)
    return int(fee_num) if fee_num else None


# 5. 문서화
for card in cards:
    # 유의사항 제외한 혜택 필터링
    filtered_benefits = [
        b for b in card["benefits"]
        if b.get("category") != "유의사항"
    ]

    # summary_box에서 연회비 및 브랜드 추출
    summary = card.get("summary_box", "")
    annual_fee_kr = re.search(r"국내전용\s*([\d,]+원)", summary)
    annual_fee_int = re.search(r"해외겸용\s*([\d,]+원)", summary)
    # 브랜드 전체 추출 및 정규화
    brand_matches = re.findall(r"(Mastercard|Visa|UnionPay|JCB|Amex)", summary, re.IGNORECASE)
    brand_matches = list(set([b.capitalize() for b in brand_matches]))  # 중복 제거 + 대소문자 정리

     # 메타데이터 구성
    metadata = {
       "card_id": card["id"],
       "name": card["name"],
       "brand": card["brand"],
       "annual_fee_domestic": parse_fee(annual_fee_kr.group(1)) if annual_fee_kr else None,
       "annual_fee_global": parse_fee(annual_fee_int.group(1)) if annual_fee_int else None,
       "global_brand": brand_matches[0] if brand_matches else None,
       "all_global_brands": brand_matches
    }

    # 카드 정보 텍스트 조립
    text = (
        f"카드명: {card['name']}\n"
        f"카드사: {card['brand']}\n"
        f"연회비(국내): {metadata['annual_fee_domestic']}원\n"
        f"연회비(해외): {metadata['annual_fee_global']}원\n"
        f"브랜드: {metadata.get('global_brand', '')}\n"
        "혜택:\n" +
        "\n".join(f"- {b['category']}: {b['details']}" for b in filtered_benefits)
    )

    docs.append(Document(page_content=text, metadata=metadata))

# 6. 임베딩 생성
texts = [doc.page_content for doc in docs]
vectors = embedder.embed_documents(texts, chunk_size=80)

# 7. Pinecone 업서트 준비
def clean_metadata(meta: dict) -> dict:
    """None이 포함된 메타데이터 값을 빈 문자열 등으로 교체"""
    return {
        k: ("" if v is None else v)
        for k, v in meta.items()
        if v is not None or isinstance(v, (str, int, float, bool, list))
    }

# 7. upsert용 페이로드 리스트 만들기
items = []
for doc, vec in zip(docs, vectors):
    clean_meta = clean_metadata(doc.metadata)
    items.append({
        "id": str(doc.metadata["card_id"]),
        "values": vec,
        "metadata": clean_meta
    })


# 8. Pinecone 업서트 (배치)
batch_size = 20
for i in range(0, len(items), batch_size):
    batch = items[i : i + batch_size]
    pinecone_index.upsert(vectors=batch)
    print(f"✔️ {i//batch_size+1}번째 배치 업서트 ({len(batch)}개) 완료")


✔️ 1번째 배치 업서트 (20개) 완료
✔️ 2번째 배치 업서트 (20개) 완료
✔️ 3번째 배치 업서트 (20개) 완료
✔️ 4번째 배치 업서트 (20개) 완료
✔️ 5번째 배치 업서트 (20개) 완료
✔️ 6번째 배치 업서트 (20개) 완료
✔️ 7번째 배치 업서트 (20개) 완료
✔️ 8번째 배치 업서트 (20개) 완료
✔️ 9번째 배치 업서트 (20개) 완료
✔️ 10번째 배치 업서트 (20개) 완료
✔️ 11번째 배치 업서트 (20개) 완료
✔️ 12번째 배치 업서트 (20개) 완료
✔️ 13번째 배치 업서트 (20개) 완료
✔️ 14번째 배치 업서트 (20개) 완료
✔️ 15번째 배치 업서트 (20개) 완료
✔️ 16번째 배치 업서트 (20개) 완료
✔️ 17번째 배치 업서트 (20개) 완료
✔️ 18번째 배치 업서트 (20개) 완료
✔️ 19번째 배치 업서트 (20개) 완료
✔️ 20번째 배치 업서트 (20개) 완료
✔️ 21번째 배치 업서트 (20개) 완료
✔️ 22번째 배치 업서트 (20개) 완료
✔️ 23번째 배치 업서트 (20개) 완료
✔️ 24번째 배치 업서트 (20개) 완료
✔️ 25번째 배치 업서트 (20개) 완료
✔️ 26번째 배치 업서트 (20개) 완료
✔️ 27번째 배치 업서트 (20개) 완료
✔️ 28번째 배치 업서트 (20개) 완료
✔️ 29번째 배치 업서트 (20개) 완료
✔️ 30번째 배치 업서트 (20개) 완료
✔️ 31번째 배치 업서트 (20개) 완료
✔️ 32번째 배치 업서트 (20개) 완료
✔️ 33번째 배치 업서트 (20개) 완료
✔️ 34번째 배치 업서트 (20개) 완료
✔️ 35번째 배치 업서트 (20개) 완료
✔️ 36번째 배치 업서트 (20개) 완료
✔️ 37번째 배치 업서트 (20개) 완료
✔️ 38번째 배치 업서트 (20개) 완료
✔️ 39번째 배치 업서트 (20개) 완료
✔️ 40번째 배치 업서트 (20개) 완료
✔️ 41번째 배치 업서트 (20개) 완료
✔️ 42번째 배치 업서트 (20개) 완료
✔